In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/val/val_week1_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/test/test_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/clean_data_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/val/val_week1_2_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/test/test_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/clean_data_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw

## 1. Import librabries

In [2]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.utils import to_categorical
import xgboost as xgb
from kerastuner.tuners import RandomSearch
import os
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.keras.utils import to_categorical

/tmp/ipykernel_23/3491627995.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


## 2. Load dataset

In [3]:
five_fold_files = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_5.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_5.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_5.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_5.csv"
    ]
}

file_validataion = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/val/val_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/val/val_week1_2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/val/val_week1_2_3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/val/val_week1_2_3_4.csv"
    ]
}

file_test = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/test/test_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/test/test_week2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/test/test_week3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/test/test_week4.csv"
    ]
}

## Mô hình FCNN

## Dự đoán các week  với best parameters

In [4]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
import shutil

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    

    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)

    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

    # Xây dựng mô hình
    def build_model(hp):
        model = tf.keras.Sequential()
        num_layers = hp.Int('num_layers', min_value=2, max_value=5, step=1)

        for i in range(num_layers):
            model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
                                   activation='relu'))
            model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))

        # Lớp đầu ra
        model.add(layers.Dense(5, activation='softmax'))

        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        return model

    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'fcnn_tuning_week{week_number}'
    )

    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)

    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    "week1": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/clean_data_week1.csv",
    "week2": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/clean_data_week2.csv",
    "week3": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/clean_data_week3.csv",
    "week4": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/clean_data_week4.csv"
}

file_validataion = {
    "week1": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/val/val_week1.csv",
    "week2": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/val/val_week1_2.csv",
    "week3": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/val/val_week1_2_3.csv",
    "week4": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/val/val_week1_2_3_4.csv"
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validataion["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validataion["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validataion["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validataion["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 01m 07s]
val_accuracy: 0.6113483905792236

Best val_accuracy So Far: 0.9768151044845581
Total elapsed time: 00h 09m 45s
Best Parameters for Week 1:
num_layers: 3
units_0: 160
dropout_0: 0.2
units_1: 128
dropout_1: 0.4
learning_rate: 0.0010624691555580386
units_2: 64
dropout_2: 0.2
units_3: 96
dropout_3: 0.2
units_4: 192
dropout_4: 0.1

Best Parameters for Week 2:
num_layers: 4
units_0: 64
dropout_0: 0.30000000000000004
units_1: 160
dropout_1: 0.30000000000000004
learning_rate: 0.0002972054749315651
units_2: 128
dropout_2: 0.1
units_3: 192
dropout_3: 0.30000000000000004
units_4: 256
dropout_4: 0.30000000000000004

Best Parameters for Week 3:
num_layers: 2
units_0: 128
dropout_0: 0.30000000000000004
units_1: 32
dropout_1: 0.1
learning_rate: 0.000580266006484294
units_2: 64
dropout_2: 0.4
units_3: 224
dropout_3: 0.1
units_4: 224
dropout_4: 0.5

Best Parameters for Week 4:
num_layers: 3
units_0: 160
dropout_0: 0.1
units_1: 128
dropout_1: 0.4
learning_rate: 0.00045428

In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_model(params):
    model = tf.keras.Sequential()
    
    # Thêm các lớp ẩn dựa trên số lượng lớp
    for i in range(params['num_layers']):
        units = params.get(f'units_{i}', 32)  # Mặc định là 32 nếu không có
        dropout = params.get(f'dropout_{i}', 0.2)  # Mặc định là 0.2 nếu không có
        model.add(layers.Dense(units=units, activation='relu'))
        model.add(layers.Dropout(rate=dropout))
    
    # Lớp đầu ra
    model.add(layers.Dense(5, activation='softmax'))  # 5 lớp output
    
    # Compile mô hình
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model

In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)
        
        # Xây dựng mô hình với tham số tốt nhất
        model = build_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["confusion_matrices"].append(conf_matrix)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.mean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.mean(week_results["f1_score_per_label"], axis=0)
    average_confusion_matrix = np.mean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    
    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    
    # In kết quả
    print("\n=== Average Precision, Recall, F1-Score per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)

    # Lưu kết quả tuần vào kết quả tổng quát
    week_results["average_train_times"] = average_train_time
    week_results["average_test_times"] = average_test_time
    week_results["average_metrics_df"] = metrics_df
    week_results["average_confusion_matrix"] = confusion_df
    overall_results_5folds.append(week_results)



Processing week1 with best parameters...
best parameters for week1: {'num_layers': 3, 'units_0': 160, 'dropout_0': 0.2, 'units_1': 128, 'dropout_1': 0.4, 'learning_rate': 0.0010624691555580386, 'units_2': 64, 'dropout_2': 0.2, 'units_3': 96, 'dropout_3': 0.2, 'units_4': 192, 'dropout_4': 0.1}
Fold 1: Using file /kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.5484 - loss: 1.6901 - val_accuracy: 0.6142 - val_loss: 1.3445
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5323 - loss: 0.7680 - val_accuracy: 0.6115 - val_loss: 0.1474
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5791 - loss: 0.1581 - val_accuracy: 0.6100 - val_loss: 0.1375
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6070 - loss: 0.1421 - val_accuracy: 0.6115 - val_loss: 0.1400
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 12s 22ms/step - accuracy: 0.5496 - loss: 1.7507 - val_accuracy: 0.5951 - val_loss: 1.3780
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5741 - loss: 1.1343 - val_accuracy: 0.5722 - val_loss: 0.1513
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5584 - loss: 0.1576 - val_accuracy: 0.6115 - val_loss: 0.1373
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5929 - loss: 0.1442 - val_accuracy: 0.6115 - val_loss: 0.1341
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6120 - loss: 0.1398 - val_accuracy: 0.6115 - val_loss: 0.1322
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6115 - loss: 0.1391 - val_accuracy: 0.6115 - val_loss: 0.1359
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6156 - loss: 0.1356 - val_accuracy: 0.6115 - val_loss: 0.1340
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6073 - loss: 0.1399 - val_accuracy: 0.6115 - va

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.5647 - loss: 1.7015 - val_accuracy: 0.6114 - val_loss: 1.5113
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6076 - loss: 1.5564 - val_accuracy: 0.6114 - val_loss: 1.5660
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6057 - loss: 1.5656 - val_accuracy: 0.6175 - val_loss: 1.4739
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5915 - loss: 1.3000 - val_accuracy: 0.6205 - val_loss: 0.1446
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5609 - loss: 0.1597 - val_accuracy: 0.6110 - val_loss: 0.1347
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6048 - loss: 0.1420 - val_accuracy: 0.6114 - val_loss: 0.1372
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6060 - loss: 0.1365 - val_accuracy: 0.6110 - val_loss: 0.1369
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6116 - loss: 0.1310 - val_accuracy: 0.6114 - va

...

328/328 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.4822 - loss: 1.9473 - val_accuracy: 0.6056 - val_loss: 1.2160
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5373 - loss: 0.9671 - val_accuracy: 0.6114 - val_loss: 0.2189
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6018 - loss: 0.2119 - val_accuracy: 0.6114 - val_loss: 0.1823
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6060 - loss: 0.1829 - val_accuracy: 0.6114 - val_loss: 0.1667
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6089 - loss: 0.1664 - val_accuracy: 0.6114 - val_loss: 0.1593
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6096 - loss: 0.1600 - val_accuracy: 0.6114 - val_loss: 0.1556
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6102 - loss: 0.1550 - val_accuracy: 0.6114 - val_loss: 0.1538
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6183 - loss: 0.1517 - val_accuracy: 0.6114 - va

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.3566 - loss: 2.4675 - val_accuracy: 0.6198 - val_loss: 1.3658
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5613 - loss: 1.1642 - val_accuracy: 0.6114 - val_loss: 0.1833
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5986 - loss: 0.2119 - val_accuracy: 0.6114 - val_loss: 0.1770
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6116 - loss: 0.1792 - val_accuracy: 0.6114 - val_loss: 0.1575
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6022 - loss: 0.1602 - val_accuracy: 0.6114 - val_loss: 0.1361
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6148 - loss: 0.1455 - val_accuracy: 0.6114 - val_loss: 0.1339
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6126 - loss: 0.1407 - val_accuracy: 0.6106 - val_loss: 0.1322
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6068 - loss: 0.1430 - val_accuracy: 0.6117 - val

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.5584 - loss: 1.7048 - val_accuracy: 0.6182 - val_loss: 1.5060
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6089 - loss: 1.4984 - val_accuracy: 0.6220 - val_loss: 1.3481
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5889 - loss: 1.3284 - val_accuracy: 0.4832 - val_loss: 0.1889
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4972 - loss: 0.2481 - val_accuracy: 0.6114 - val_loss: 0.1807
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6088 - loss: 0.1767 - val_accuracy: 0.6114 - val_loss: 0.1359
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6138 - loss: 0.1444 - val_accuracy: 0.6114 - val_loss: 0.1336
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6125 - loss: 0.1392 - val_accuracy: 0.6114 - val_loss: 0.1347
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6129 - loss: 0.1380 - val_accuracy: 0.6114 - val

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_confusion_matrix = week_result["average_confusion_matrix"]

    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print("\nAverage Precision, Recall, F1-Score per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Train Time: 43.0070 seconds
Average Test Time: 0.4000 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.659017        0.655000          0.654999
1      1           0.000000        0.000000          0.000000
2      2           0.746665        0.415928          0.529182
3      3           0.651072        0.488623          0.555127
4      4           0.841510        0.946492          0.890778

Average Confusion Matrix:
       0    1     2     3       4
0  393.0  0.0  11.8  23.6   171.6
1   54.6  0.0   1.6   3.8    27.6
2   46.4  0.0  68.4   8.4    41.2
3   36.4  0.0   3.0  81.6    46.0
4   69.8  0.0   7.6   8.4  1517.6

=== Results for week2 ===
Average Train Time: 52.0312 seconds
Average Test Time: 0.4159 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.658208        0.641333  

## Kiểm tra trên tập test

In [11]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_test = test_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=5)
    
    # Lấy tham số tốt nhất cho tuần
    params = best_params[f"week{week_num}"].values
    
    # Xây dựng mô hình
    model = build_model(params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_res, y_train_res, epochs=50, validation_split=0.1, batch_size=32, verbose=1)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    
    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix
    })
    
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [12]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.2718 - loss: 1.5339 - val_accuracy: 0.0000e+00 - val_loss: 0.3771
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3304 - loss: 0.2342 - val_accuracy: 0.1532 - val_loss: 0.3299
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3608 - loss: 0.2261 - val_accuracy: 0.3579 - val_loss: 0.2992
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3931 - loss: 0.2180 - val_accuracy: 0.3642 - val_loss: 0.2933
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3985 - loss: 0.2140 - val_accuracy: 0.3714 - val_loss: 0.3057
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4116 - loss: 0.2090 - val_accuracy: 0.5206 - val_loss: 0.2476
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4183 - loss: 0.2053 - val_accuracy: 0.4642 - val_loss: 0.2669
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - ac

In [13]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.2637 - loss: 1.5263 - val_accuracy: 0.0000e+00 - val_loss: 0.2777
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3331 - loss: 0.2483 - val_accuracy: 0.0000e+00 - val_loss: 0.3052
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3902 - loss: 0.2251 - val_accuracy: 0.3821 - val_loss: 0.2598
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4234 - loss: 0.2184 - val_accuracy: 0.4096 - val_loss: 0.2718
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4475 - loss: 0.2127 - val_accuracy: 0.4131 - val_loss: 0.2662
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4669 - loss: 0.2076 - val_accuracy: 0.4183 - val_loss: 0.2435
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4844 - loss: 0.2029 - val_accuracy: 0.4228 - val_loss: 0.2328
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step 

In [14]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.2698 - loss: 2.2078 - val_accuracy: 0.0000e+00 - val_loss: 0.3234
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2363 - loss: 0.2554 - val_accuracy: 0.0000e+00 - val_loss: 0.3879
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2718 - loss: 0.2463 - val_accuracy: 0.0000e+00 - val_loss: 0.3975
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2913 - loss: 0.2398 - val_accuracy: 0.0000e+00 - val_loss: 0.3504
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.3044 - loss: 0.2350 - val_accuracy: 0.1100 - val_loss: 0.3462
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.3211 - loss: 0.2289 - val_accuracy: 0.3921 - val_loss: 0.3427
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3443 - loss: 0.2237 - val_accuracy: 0.4171 - val_loss: 0.3170
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 1m

In [15]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.2714 - loss: 1.2732 - val_accuracy: 0.0000e+00 - val_loss: 0.3430
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2988 - loss: 0.2409 - val_accuracy: 0.0000e+00 - val_loss: 0.3307
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3130 - loss: 0.2331 - val_accuracy: 0.0000e+00 - val_loss: 0.3948
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3259 - loss: 0.2275 - val_accuracy: 0.3921 - val_loss: 0.3428
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3373 - loss: 0.2235 - val_accuracy: 0.4428 - val_loss: 0.3470
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3531 - loss: 0.2191 - val_accuracy: 0.4500 - val_loss: 0.3081
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3676 - loss: 0.2129 - val_accuracy: 0.3929 - val_loss: 0.2880
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/s

In [16]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 102.39 seconds
  Test Time: 0.53 seconds
  Precision: [0.79955947 0.28947368 0.79032258 0.90740741 0.94246862]
  Recall: [0.968      0.61111111 0.47572816 0.46666667 0.89830508]
  F1-Score: [0.87575392 0.39285714 0.59393939 0.6163522  0.91985707]
  Confusion Matrix:
[[363   6   1   0   5]
 [  0  33   2   0  19]
 [  0  44  49   5   5]
 [  0  23   7  49  26]
 [ 91   8   3   0 901]]
Week 2:
  Train Time: 115.97 seconds
  Test Time: 0.56 seconds
  Precision: [0.9295393  0.53191489 0.89230769 1.         0.97230465]
  Recall: [0.91466667 0.92592593 0.5631068  0.96190476 0.98005982]
  F1-Score: [0.92204301 0.67567568 0.69047619 0.98058252 0.97616683]
  Confusion Matrix:
[[343   5   1   0  26]
 [  2  50   2   0   0]
 [  6  37  58   0   2]
 [  0   1   3 101   0]
 [ 18   1   1   0 983]]
Week 3:
  Train Time: 94.04 seconds
  Test Time: 0.48 seconds
  Precision: [0.99468085 0.67605634 0.92391304 1.         1.        ]
  Recall: [0.997333

In [17]:
# import tensorflow as tf
# from tensorflow.keras import layers
# import keras.backend as K
# import pandas as pd
# import numpy as np
# from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
# from imblearn.over_sampling import SMOTE
# import time

# # Định nghĩa Focal Loss
# def focal_loss(gamma=2., alpha=0.25):
#     def focal_loss_fixed(y_true, y_pred):
#         y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1. - K.epsilon())
#         cross_entropy = -y_true * tf.math.log(y_pred)
#         loss = alpha * tf.math.pow(1 - y_pred, gamma) * cross_entropy
#         return tf.reduce_sum(loss, axis=-1)
#     return focal_loss_fixed


# # Xây dựng mô hình
# def build_model(params):
#     model = tf.keras.Sequential()
    
#     # Thêm các lớp ẩn dựa trên số lượng lớp
#     for i in range(params['num_layers']):
#         units = params.get(f'units_{i}', 32)  # Mặc định là 32 nếu không có
#         dropout = params.get(f'dropout_{i}', 0.2)  # Mặc định là 0.2 nếu không có
#         model.add(layers.Dense(units=units, activation='relu'))
#         model.add(layers.Dropout(rate=dropout))
    
#     # Lớp đầu ra
#     model.add(layers.Dense(5, activation='softmax'))  # 5 lớp output
    
#     # Compile mô hình
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
#         loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
#         metrics=['accuracy']
#     )
#     return model

# # Load dữ liệu
# train_data = pd.read_csv("/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/clean_data_week1.csv")
# test_data = pd.read_csv("/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/test/test_week1.csv")

# # Tách đặc trưng và nhãn
# X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
# y_train = train_data["classification_encoded"]

# X_test = test_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
# y_test = test_data["classification_encoded"]

# # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
# oversampler = SMOTE(sampling_strategy='auto', random_state=42)
# X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)

# # One-hot encode nhãn
# y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
# y_test = tf.keras.utils.to_categorical(y_test, num_classes=5)

# # Định nghĩa tham số tốt nhất
# best_params_week1 = {
#     "num_layers": 2,
#     "units_0": 64,
#     "units_1": 32,
#     "dropout_0": 0.3,
#     "dropout_1": 0.2,
#     "learning_rate": 0.001,
#     "gamma": 2.0,
#     "alpha": 0.25
# }

# best_params = {"week1": best_params_week1}

# # Xây dựng mô hình với tham số tốt nhất
# params = best_params["week1"]
# model = build_model(params)

# # Huấn luyện mô hình
# start_train = time.time()
# model.fit(X_train_res, y_train_res, epochs=50, validation_split=0.1, batch_size=32, verbose=1)
# end_train = time.time()

# # Kiểm thử mô hình
# start_test = time.time()
# y_pred = model.predict(X_test)
# end_test = time.time()

# # Tính thời gian huấn luyện và kiểm thử
# train_time = end_train - start_train
# test_time = end_test - start_test

# # Đánh giá mô hình
# y_pred_classes = y_pred.argmax(axis=1)
# y_test_classes = y_test.argmax(axis=1)

# precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
# conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

# # Hiển thị kết quả
# labels = np.unique(y_test_classes)
# metrics_df = pd.DataFrame({
#     "Label": labels,
#     "Precision": precision,
#     "Recall": recall,
#     "F1-Score": f1
# })
# confusion_df = pd.DataFrame(conf_matrix, index=labels, columns=labels)

# print("\n=== Precision, Recall, F1-Score per Label ===")
# print(metrics_df)
# print("\n=== Confusion Matrix ===")
# print(confusion_df)

# print(f"\nTrain Time: {train_time:.2f} seconds")
# print(f"Test Time: {test_time:.2f} seconds")
